In [16]:
PROJECT_NAME = "Supermarket Hot Deals Thompson Experiment"
NUM_PRODUCTS = 6   
PRODUCTS_PER_PAGE = 3

with Session(engine) as session:
    project = Project(
        project_name=PROJECT_NAME,
        project_description="Auto-created test for modeling",
        number_of_bandits=NUM_PRODUCTS
    )
    session.add(project)
    session.commit()
    print(f"Created project_id: {project.project_id}")

    project_id = project.project_id 

    for i in range(NUM_PRODUCTS):
        bandit = Bandit(
            project_id=project_id,
            bandit_name=f"Product {i+1}",
            alpha=1,
            beta=1,
            n=0,
            number_of_success=0,
            number_of_failures=0
        )
        session.add(bandit)
    session.commit()

def get_page_recommendations(project_id, page_num=0):
    with Session(engine) as session:
        bandits = session.query(Bandit).filter(Bandit.project_id == project_id).all()
        samples = [np.random.beta(b.alpha, b.beta) for b in bandits]
        sorted_bandits = [b for _, b in sorted(zip(samples, bandits), key=lambda pair: pair[0], reverse=True)]
        start = page_num * PRODUCTS_PER_PAGE
        end = start + PRODUCTS_PER_PAGE
        return sorted_bandits[start:end]

for p in range(2):
    print(f"\nRecommended products on page {p+1}:")
    recs = get_page_recommendations(project_id=project_id, page_num=p)
    for b in recs:
        print(f"{b.bandit_name} (alpha={b.alpha}, beta={b.beta})")

def simulate_click(bandit: Bandit, clicked: bool):
    with Session(engine) as session:
        db_bandit = session.query(Bandit).filter(Bandit.bandit_id == bandit.bandit_id).first()
        if clicked:
            db_bandit.alpha += 1
            db_bandit.number_of_success += 1
        else:
            db_bandit.beta += 1
            db_bandit.number_of_failures += 1
        db_bandit.n += 1
        session.commit()

simulate_click(recs[0], clicked=True)
simulate_click(recs[1], clicked=False)

Created project_id: 5

Recommended products on page 1:
Product 1 (alpha=1.0, beta=1.0)
Product 3 (alpha=1.0, beta=1.0)
Product 5 (alpha=1.0, beta=1.0)

Recommended products on page 2:
Product 3 (alpha=1.0, beta=1.0)
Product 5 (alpha=1.0, beta=1.0)
Product 6 (alpha=1.0, beta=1.0)
